In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Prose/Mathieu/Benter-Project

/home/mathieu/Prose/Mathieu/Benter-Project


In [8]:
%matplotlib inline

import datetime as dt
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
from itertools import combinations
import tensorflow as tf
import functools
from scipy.stats import rankdata
import scipy
import json

from utils import import_data
from winning_validation import errors
from winning_validation import r_squared
from winning_horse_models import sklearn
from winning_horse_models.dl_shared_layers import LogisticRegressionModel, DLSharedLayersModel, DLLayersGeneratorModel
from winning_horse_models.xgboost import XGBoostWinningModel
from winning_horse_models.catboost import CatboostWinningModel
from winning_horse_models.lgbm import LGBMWinningModel
from training_procedures import sequential_training, flattened_training
from constants import Sources, SplitSets, XFormats, YFormats
from catboost import CatBoostClassifier
from utils import preprocess
from sklearn.feature_selection import SequentialFeatureSelector

from database.setup import create_sqlalchemy_session
from models.race import Race
from models.runner import Runner
from sklearn.linear_model import SGDClassifier
tqdm.pandas()

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

/home/mathieu/.pyenv/versions/3.7.9/envs/benter-project_venv/lib/python3.7/site-packages/tqdm/std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
SOURCE = Sources.UNIBET
N_FEATURES = preprocess.get_n_preprocessed_feature_columns(source=SOURCE)

# DLSharedLGBMWinningModel

In [ ]:
%%time
%%capture --no-stdout
space = {'layers':[{'type':'Dense', 'n_units':1}]}

model =DLLayersGeneratorModel(source=SOURCE, n_features = N_FEATURES, hyperparameters=space, name='1')
    
model, _ = sequential_training.train_on_n_horses(source=SOURCE, winning_model=model, n_horses=10, n_epochs=0, start_training_at=dt.datetime.now(),
                                                 n_epochs_per_n_horses=1, verbose=True)

res=r_squared.compute_mcfadden_r_squared_on_n_horses(source=SOURCE,winning_model=model, n_horses=10, verbose=True)
print(res)

In [6]:
x, y, _ = import_data.get_races_per_horse_number(
    source=SOURCE,
    n_horses=10,
    on_split=SplitSets.TRAIN,
    x_format=XFormats.FLATTENED,
    y_format=YFormats.INDEX_FIRST,
    preprocessing=True,
)

In [10]:
%%time
model = SGDClassifier()
sfs = SequentialFeatureSelector(model, n_features_to_select=10, n_jobs=-2)
sfs.fit(x, y)
    

CPU times: user 5min 54s, sys: 14.8 s, total: 6min 9s
Wall time: 1h 48min 50s


SequentialFeatureSelector(estimator=SGDClassifier(), n_features_to_select=10,
                          n_jobs=-2)

In [12]:
sfs.support_

array([False, False, False, ..., False, False, False])

In [13]:
sfs.transform(x)

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.9999101 ,  3.395655  , -2.939271  , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.24302524, -2.939271  , -0.82762897, ...,  0.        ,
         0.        ,  0.        ],
       [-0.24302524, -0.7672963 ,  0.56515616, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01193586, -0.08234358,  0.41364065, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [15]:
model =CatBoostClassifier(**{'border_count': 76, 'depth': 6, 'l2_leaf_reg': 5.3399121437223345, 'learning_rate': 0.033673676460578644, 'verbose': 0})

In [16]:
model.fit(sfs.transform(x), y)

In [17]:
x_val, y_val, _ = import_data.get_races_per_horse_number(
    source=SOURCE,
    n_horses=10,
    on_split=SplitSets.VAL,
    x_format=XFormats.FLATTENED,
    y_format=YFormats.INDEX_FIRST,
    preprocessing=True,
)

In [18]:
y_val_pred = model.predict_proba(sfs.transform(x_val))

In [20]:
y_val_pred.shape

(3182, 10)

In [29]:
y_val.reshape((-1, 1)).shape

(3182, 1)

In [46]:
1+np.sum(np.log(np.choose(y_val.astype(np.int32), y_val_pred.T)))/(x_val.shape[0]*np.log(x_val.shape[1]))

0.6742278164283964

In [44]:
x_val.shape[0]

3182

In [ ]:
model =CatBoostClassifier(**{'border_count': 76, 'depth': 6, 'l2_leaf_reg': 5.3399121437223345, 'learning_rate': 0.033673676460578644, 'verbose': 0})
model.fit(x, y)
y_val_pred = model.predict_proba(x_val)
1+np.sum(np.log(np.choose(y_val.astype(np.int32), y_val_pred.T)))/(x_val.shape[0]*np.log(x_val.shape[1]))